# Improving the model
## Let's load the data

In [49]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier

df = pd.read_csv("diabetes.csv")

## Data

Firstly the data you put in makes the biggest difference to how well your network will perform. In this case this dataset has some additional information that is in the blurb that isn't in the dataset itself.

Now lets add if those ranges are within normals. Here's the normals from [Kaggle](https://www.kaggle.com/jamaltariqcheema/pima-indians-diabetes-dataset)


> Glucose: Glucose (< 140) = Normal, Glucose (140-200) = Pre-Diabetic, Glucose (> 200) = Diabetic
> BloodPressure: B.P (< 60) = Below Normal, B.P (60-80) = Normal, B.P (80-90) = Stage 1 Hypertension, B.P (90-120) = Stage 2 Hypertension, B.P (> 120) = Hypertensive Crisis
> SkinThickness: SkinThickness (< 10) = Below Normal, SkinThickness (10-30) = Normal, SkinThickness (> 30) = Above Normal
> Insulin: Insulin (< 200) = Normal, Insulin (> 200) = Above Normal
> BMI: BMI (< 18.5) = Underweight, BMI (18.5-25) = Normal, BMI (25-30) = Overweight, BMI (> 30) = Obese


In [ ]:
df["GlucoseRange"] = pd.cut(df['Glucose'], [0, 140, 200, 999], labels=[0, 1, 2])
df["BloodPressureRange"] = pd.cut(df['BloodPressure'], [0, 60, 80, 90, 120, 999], labels=[0, 1, 2, 3, 4])
df["SkinThicknessRange"] = pd.cut(df['SkinThickness'], [0, 10, 30, 999], labels=[0, 1, 2])
df["InsulinRange"] = pd.cut(df['Insulin'], [0, 200, 999], labels=[0, 1])
df["BMIRange"] = pd.cut(df['BMI'], [0, 18.5, 25, 30, 999], labels=[0, 1, 2, 3])
df

Now we pull out the data from

In [51]:
y = df.Outcome.to_numpy()
x = df.drop(columns=["Outcome"]).to_numpy()

## Now the exciting bit

Lets load up our neural net. Note that this time we are creating it in a function, and we have changed the layers a little. A good rule of thumb is

![Rule of thumb](./img.png "Rule of thumb")

In our case we have `767/(5*(13*1))`, so a bit over `11`. There's loads of advice on this fantastic post on [Stack Overflow](https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw).

In [52]:
def create_model(optimizer, kernel_initializer):
    model = tf.keras.Sequential()
    model.add(
        tf.keras.layers.Dense(
            7,

            kernel_initializer=kernel_initializer,
            activation=tf.keras.activations.relu
        )
    )
    model.add(
        tf.keras.layers.Dense(
            3,
            kernel_initializer=kernel_initializer,
            activation=tf.keras.activations.relu
        )
    )
    model.add(
        tf.keras.layers.Dense(
            1,
            kernel_initializer=kernel_initializer,
            activation=tf.keras.activations.sigmoid
        )
    )

    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=optimizer,
        metrics=[
            tf.keras.metrics.BinaryAccuracy(name="accuracy"),
        ]
    )
    return model

## Finding the rest of the parameters

We are going to use something called a KFold evaluating the model in a Grid Search. the important things to know about this is it's going to brute force its way through your parameters (in like a mathy, sort of efficient way) and try and find the best combination of inputs to create an efficient network. It also handles holding back some data for testing, so we don't need to sample a percentage of the data ourselves.

In [ ]:
classifier = KerasClassifier(build_fn=create_model)
grid = GridSearchCV(
    estimator=classifier,
    param_grid={
        "batch_size": [10, 15, 20],
        "epochs": list(range(50, 200, 50)),
        "kernel_initializer": [
            tf.keras.initializers.random_uniform,
            tf.keras.initializers.random_normal,
            tf.keras.initializers.glorot_uniform,
        ],
        "optimizer": [tf.keras.optimizers.Adam(), tf.keras.optimizers.RMSprop()]
    },
)
grid_result = grid.fit(x, y)

## Aaaand your model is done

Did you have a nice break? Get a good coffee? Awesome!

Your result should be a good 5% better than before when you have made the changes that were found. According to [Kaggle](https://www.kaggle.com/jamaltariqcheema/pima-indians-diabetes-dataset) 92% is the score to beat.

In [ ]:
grid_result.best_score_

In [ ]:
grid_result.best_params_

Now we have the best parameters for this.

## Following on

1. What other things could you include in your search?
2. How might you iterate with a grid search which minimising development time?
3. What other approaches might you take to tuning?